# Circuit simulations

Every gdsfactory component can accss its netlist `get_netlist()` and Sparameters.

You can use a circuit simulation tool such as [Simphony](https://github.com/BYUCamachoLab/simphony)

In [ ]:
import pp

c = pp.c.mzi()
pp.qp(c)

n = c.get_netlist()